# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/119000009781688
# 迭代器
- 可迭代（Iterable）：直接作用于for循环变量
- 迭代器（Iterator）:不但可以作用于for循环，还可以被next调用
- list是典型的可迭代的对象，但不是迭代器
- 通过isinstance() 判断
- iterable 和iterator可以转换
    -通过iter函数

In [1]:
# 可迭代
l =[i for i in range(10)]
#l是可迭代的，但不是迭代器
for idx in l:
    print(idx)
#range是一个迭代器
for i in range(5):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [7]:
#isinstance 案例
#判断某个变量是否是一个实例
#判断是否可迭代
from collections import Iterable
ll = [1,2,3,4,5]
print(isinstance(ll,Iterable))
from collections import Iterator
print(isinstance(ll,Iterator))

True
False


In [8]:
#iter函数
s = 'i love wangxiaojing'
print(isinstance(s,Iterable))
print(isinstance(s,Iterator))
s_iter = iter(s)
print(isinstance(s_iter,Iterable))
print(isinstance(s_iter,Iterator))

True
False
True
True


# 生成器
- generator:一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件
    - 每次调用 都产生出for循环需要的下一个元素或者
    - 如果达不到最后一个数，爆出StopIteration异常
    - 可以被next函数调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫做生成器
    - next调用函数，遇到yield返回

In [12]:
#直接使用生成器
L = [x*x for x in range(5)]#放在中括号中是列表生成器
g = (x*x for x in range(5))#放在小括号中就是一个生成器
print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [13]:
# 函数案例
def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    print("Step 4")
    return None
odd()

Step 1
Step 2
Step 3
Step 4


In [20]:
# 生成器案例
# 函数odd中，yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
# odd() 是调用生成器   
g = odd()
one = next(g)
print(one)
two = next(g)
print(two)
for i in odd():
    print(i)

Step 1
1
Step 2
2
Step 1
1
Step 2
2
Step 3
3


In [25]:
# for 循环调用生成器
def fib(max):
    n,a,b = 0,0,1 #注意写法
    while n < max:
        print(b)
        a,b = b, a+b #注意写法
        n += 1
    return 'Done'
fib(5)

1
1
2
3
5


'Done'

In [29]:
#斐波那契额数列的生成器写法
def fib(max):
    n,a,b = 0,0,1 #注意写法
    while n < max:
        yield b
        a,b = b, a+b #注意写法
        n += 1
    # 需要注意，爆出异常是的返回值 return的返回值
    return 'Done'
g=fib(5)
for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

In [30]:
ge = fib(10)
'''
生成器的典型用法是在for中使用
比较常用的典型生成器就是range
'''
for i in ge: # 在for循环中使用生成器
    print(i)

1
1
2
3
5
8
13
21
34
55


# 协程
- 历史历程
    - 3.4引入协程，用yield实现
    - 3.5 引入协程语法
    - 实现的协程比较好的包有asyncio,tornado,gevent
- 定义:协程是为了非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置站停或开始执行程序
- 从技术角度讲，携程就是一个你可以站厅执行的函数，或者干脆把协程理解成生成器
- 协程的实现:
    - yield返回
    - send调用
- 协程的四个状态
    - inspect.getgeneratorstate(...) 函数确定，该函数会返回下述字符串中的一个
    - GEN_CREATED:等待开始执行
    - GEN_RUNNING:解释器正在执行
    - GEN_SUSPENED:在yield表达式处暂停
    - GEN_CLOSED:执行结束
    - next预激（prime）
    - 代码案例v2
- 协程终止
    - 协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）
    - 终止协程的一种方式：发送某个哨符值，让协程退出，内置的None和Ellipsis等常量经常用作哨符值==
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出StopIteration异常，异常对象的value属性保存返回值
    - yield from从内部捕获StopIteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from 表达式的生成器函数
        - 委派生成器在yield from 表达式出暂停，调用方可以直接把数据发给子生成器
        - 子生成器在把产出的值发给调用方
        - 子生成器在最后，解释器会抛出StopIteration,并且把返回值附加到异常对象上
        - 案例v04 
    

In [ ]:
# 协程代码案例1
def simple_coroutine():
    print('->start')
    x = yield
    print('->received',x)
sc = simple_coroutine() #产生协程
print(1111)
#可以使用sc.send(None)，效果一样
next(sc) # 预ji激
print(2222)
sc.send("zhexiao")

In [31]:
# 案例v2 协程的状态
def simple_coroutine(a):
    print('->start')
    b = yield a
    print('->recieved',a,b)
    c = yield a+b
    print('->recieved',a,b,c)
# runc
sc = simple_coroutine(5)
aa = next(sc)
print(aa)
bb = sc.send(6)# 5,6
print(bb)
cc = sc.send(7)#5,6,7
print(cc)

->start
5
->recieved 5 6
11
->recieved 5 6 7


StopIteration: 

In [32]:
# 案例v03
def gen():
    for c in "AB":
        yield c
#list直接用协程生成器作为参数
print(list(gen()))
def gen_new():
    yield from 'AB'
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [33]:
 # 案例v04 委派生成器
from collections import namedtuple
'''
解释:
1 .外层for循环每次迭代会新建一个grouper实例，赋值给 coroutine变量，grouper是 委派生成器
2.调用next(coroutine),预激委派生成器grouper,此时进入while True循环，调用子生成器averager
3.内层for循环调用 coroutine.send(value),直接把值传递给子生成器averager.同时，当前的grouper
4 内层循环结束后，grouper实例依旧在yield from 表达式处暂停，因此，grouper函数定义体中
5.coroutine.send(None)终止averager子生成器，子生成器抛出StopIteration异常并将返回的数
'''
ResClass = namedtuple('Res','count average')
# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return ResClass(count,average)

#委派生成器
def grouper(storages,key):
    while True:
        # 获取average()返回的值
        storages[key] = yield from averager()
# 客户端代码
def client():
    progress_data={
        'boys_2':[39.0,40.8,43.2,40.8,43.1,38.6,41.4,40.6,36,3],
        'boys_1':[1.38,1.5,1.32,1.25,1.37,1.48,1.25,1.49,1.46]
    }
    storages = {}
    for k,v in progress_data.items():
        #获得协程
        coroutine = grouper(storages,k)
        #预激协程
        next(coroutine)
        #发送数据到协程
        for dt in v:
            coroutine.send(dt)
        # 终止协程
        coroutine.send(None)
    print(storages)
client()        

{'boys_2': Res(count=10, average=36.65), 'boys_1': Res(count=9, average=1.3888888888888888)}
